In [28]:
!pip install autocorrect

In [29]:
import pandas as pd
import gensim
from gensim.models import Word2Vec
from gensim.models import fasttext
from autocorrect import Speller
import nltk
from nltk.stem import WordNetLemmatizer
import re
from sklearn.utils import shuffle

In [30]:
import numpy as np
from seaborn import countplot
from pyemd import emd

In [31]:
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('omw-1.4')
from nltk.corpus import stopwords

In [32]:
lemmatizer = WordNetLemmatizer()
spell = Speller(lang='en')

In [33]:
def classify(test_review,k):
    all_distances = np.array([dist(test_review, train_review) for train_review in train_data])
    sorted_idx = all_distances.argsort()
    n_closest = train_labels[sorted_idx][:k]
    unique, pos = np.unique(n_closest,return_inverse=True)
    counts = np.bincount(pos)
    maxpos = counts.argmax()
    prediction= unique[maxpos]
    return prediction

In [34]:
#data_loc = 'data/'

In [35]:
data1 = pd.read_csv('winemag-data_first150k.csv', index_col=0)
data2 = pd.read_csv('winemag-data-130k-v2.csv', index_col=0)

In [36]:
#Panen andmed kokku
dataset = pd.concat([data1,data2])

In [37]:
dataset = shuffle(dataset, random_state=123)

In [38]:
dataset = dataset.reset_index(drop=True)
dataset

,country,description,designation,points,price,province,region_1,region_2,variety,winery,taster_name,taster_twitter_handle,title
0,US,"Sparkling wine is all about texture, and the m...",Brut,85,22.0,California,Russian River Valley,Sonoma,Sparkling Blend,River Road,NaN,NaN,NaN
1,France,"This fruity wine, free from any oak aging, rev...",NaN,85,15.0,Burgundy,Mâcon-Villages,NaN,Chardonnay,Labouré-Roi,Roger Voss,@vossroger,Labouré-Roi 2015 Mâcon-Villages
2,US,"A rich, ripe and complex Cabernet, with classi...",Estate Bottled,90,35.0,California,Napa Valley,Napa,Cabernet Sauvignon,St. Supéry,NaN,NaN,NaN
3,Italy,Here's a cheerful white blend from Tuscany (wi...,Fonte delle Donne,87,18.0,Tuscany,Toscana,NaN,White Blend,Fattoria Fibbiano,NaN,NaN,NaN
4,New Zealand,"This medium-weight, supple Pinot Noir is a bit...",Estate Series,87,24.0,Central Otago,NaN,NaN,Pinot Noir,Matua Valley,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
280896,Portugal,"Ripe, rich and creamy, this is a dense wine fr...",Reserva,88,12.0,Douro,NaN,NaN,Portuguese White,Borges,Roger Voss,@vossroger,Borges 2015 Reserva White (Douro)
280897,US,"Pretty much a copy of the '03, which I found t...",Monte Rosso Vineyard Gnarly Vine,84,50.0,California,Sonoma Valley,Sonoma,Zinfandel,Louis M. Martini,NaN,NaN,NaN
280898,US,"The blend is Zinfandel, Syrah, Cabernet Sauvig...",Essential,83,11.0,California,California,California Other,Red Blend,Bogle,NaN,NaN,NaN
280899,US,"The aromas of plum, blackberry, vanilla, dried...",NaN,87,14.0,Washington,Columbia Valley (WA),Columbia Valley,Syrah,Waterbrook,Sean P. Sullivan,@wawinereport,Waterbrook 2014 Syrah (Columbia Valley (WA))


In [39]:
dataset.iloc[205000:]

,country,description,designation,points,price,province,region_1,region_2,variety,winery,taster_name,taster_twitter_handle,title
205000,US,"This is straightforward and easy, with sweet M...",NaN,85,16.0,California,Napa Valley,Napa,Sauvignon Blanc,Lola Kay,NaN,NaN,Lola Kay 2011 Sauvignon Blanc (Napa Valley)
205001,Italy,"Upfront and generous, this is a bright and foo...",Donia,86,18.0,Piedmont,Barbera d'Alba,NaN,Barbera,Ponchione Maurizio,NaN,NaN,Ponchione Maurizio 2007 Donia (Barbera d'Alba)
205002,Italy,"Robust but informal, this opens with aromas of...",Rosso,88,12.0,Southern Italy,Cirò,NaN,Gaglioppo,Librandi,Kerin O’Keefe,@kerinokeefe,Librandi 2013 Rosso (Cirò)
205003,Spain,"Dusty berry, resin and bell-pepper aromas sugg...",Crianza,89,20.0,Northern Spain,Ribera del Duero,NaN,Tempranillo,Tamaral,NaN,NaN,NaN
205004,Austria,"Beautiful crushed red cherry and lifted, polis...",Vom Kiesel,92,NaN,Burgenland,NaN,NaN,St. Laurent,Strehn,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
280896,Portugal,"Ripe, rich and creamy, this is a dense wine fr...",Reserva,88,12.0,Douro,NaN,NaN,Portuguese White,Borges,Roger Voss,@vossroger,Borges 2015 Reserva White (Douro)
280897,US,"Pretty much a copy of the '03, which I found t...",Monte Rosso Vineyard Gnarly Vine,84,50.0,California,Sonoma Valley,Sonoma,Zinfandel,Louis M. Martini,NaN,NaN,NaN
280898,US,"The blend is Zinfandel, Syrah, Cabernet Sauvig...",Essential,83,11.0,California,California,California Other,Red Blend,Bogle,NaN,NaN,NaN
280899,US,"The aromas of plum, blackberry, vanilla, dried...",NaN,87,14.0,Washington,Columbia Valley (WA),Columbia Valley,Syrah,Waterbrook,Sean P. Sullivan,@wawinereport,Waterbrook 2014 Syrah (Columbia Valley (WA))


In [40]:
dataset = dataset.iloc[:205000]

In [41]:
#Vaatan, mis toimub
dataset.head()

,country,description,designation,points,price,province,region_1,region_2,variety,winery,taster_name,taster_twitter_handle,title
0,US,"Sparkling wine is all about texture, and the m...",Brut,85,22.0,California,Russian River Valley,Sonoma,Sparkling Blend,River Road,NaN,NaN,NaN
1,France,"This fruity wine, free from any oak aging, rev...",NaN,85,15.0,Burgundy,Mâcon-Villages,NaN,Chardonnay,Labouré-Roi,Roger Voss,@vossroger,Labouré-Roi 2015 Mâcon-Villages
2,US,"A rich, ripe and complex Cabernet, with classi...",Estate Bottled,90,35.0,California,Napa Valley,Napa,Cabernet Sauvignon,St. Supéry,NaN,NaN,NaN
3,Italy,Here's a cheerful white blend from Tuscany (wi...,Fonte delle Donne,87,18.0,Tuscany,Toscana,NaN,White Blend,Fattoria Fibbiano,NaN,NaN,NaN
4,New Zealand,"This medium-weight, supple Pinot Noir is a bit...",Estate Series,87,24.0,Central Otago,NaN,NaN,Pinot Noir,Matua Valley,NaN,NaN,NaN


In [42]:
desc = dataset.description

In [43]:
reviews_lower = [x.lower() for x in desc]

In [44]:
reviews_lower[0]

'sparkling wine is all about texture, and the mouthfeel on this chardonnay-based bubbly lacks refinement. the bubbles are scoury and the finish is rough. still, it has rich citrus, yeast, honey and pear flavors.'

In [45]:
#Puhastan teksti sodist, aga võite vaielda, et äkki ikkagi on vaja mingit tüüpi arve säilitada (nt aastaarvud)
cleaned_reviews = [re.sub('[^a-zA-Z]', ' ', x) for x in reviews_lower]
cleaned_reviews = [re.sub(r'\s+', ' ', x) for x in cleaned_reviews]

In [46]:
cleaned_reviews[0]

'sparkling wine is all about texture and the mouthfeel on this chardonnay based bubbly lacks refinement the bubbles are scoury and the finish is rough still it has rich citrus yeast honey and pear flavors '

In [47]:
reviews = [nltk.sent_tokenize(x) for x in cleaned_reviews]

In [48]:
reviews = [y for x in reviews for y in x]

In [49]:
reviews[0]

'sparkling wine is all about texture and the mouthfeel on this chardonnay based bubbly lacks refinement the bubbles are scoury and the finish is rough still it has rich citrus yeast honey and pear flavors'

In [50]:
words = [nltk.word_tokenize(x) for x in reviews]

In [51]:
stop_words = stopwords.words('english')

In [52]:
for i in range(len(words)):
    words[i] = [lemmatizer.lemmatize(w) for w in words[i] if w not in stop_words and len(lemmatizer.lemmatize(w)) > 1]
    # words[i] = [w for w in words[i] if w not in stop_words]
# words = [[w for w in words[i] if w not in stop_words] for i in range(len(words))]

In [53]:
#Põhimõtteliselt on korpus valmis
words[0]

['sparkling',
 'wine',
 'texture',
 'mouthfeel',
 'chardonnay',
 'based',
 'bubbly',
 'lack',
 'refinement',
 'bubble',
 'scoury',
 'finish',
 'rough',
 'still',
 'rich',
 'citrus',
 'yeast',
 'honey',
 'pear',
 'flavor']

In [54]:
# Mudel, aga epohhe võiks rohkem olla (100)
# model = gensim.models.Word2Vec(words, vector_size = 200 , window = 4 , min_count = 1,
#                                seed = 1000, hs = 0, negative = 10,workers=16, epochs = 25)
model = Word2Vec(words, vector_size = 200 , window = 4 , min_count = 1,
                               seed = 1000, hs = 0, negative = 10,workers=16, epochs = 50)

In [55]:
model.save("word2vec1.model")

In [56]:
model = Word2Vec.load("word2vec1.model")


In [57]:
#Kümme kõige lähedasemat sõna sõnale "magus"
model.wv.most_similar('sweet')

[('sweetened', 0.5190536379814148),
 ('sweetness', 0.45396143198013306),
 ('semisweet', 0.44037604331970215),
 ('sugary', 0.4317525029182434),
 ('jam', 0.4268885850906372),
 ('flavor', 0.42610424757003784),
 ('candied', 0.42477771639823914),
 ('caramelized', 0.41631636023521423),
 ('sugared', 0.4126093089580536),
 ('vanilla', 0.39884471893310547)]

In [58]:
#Kümme kõige lähedasemat sõna sõnale "kuiv"
model.wv.most_similar('dry')

[('dryness', 0.5310472249984741),
 ('drily', 0.48134157061576843),
 ('scour', 0.37994417548179626),
 ('crackingly', 0.3740661144256592),
 ('semidry', 0.36478203535079956),
 ('dryish', 0.35937315225601196),
 ('brisk', 0.34867554903030396),
 ('drier', 0.33254358172416687),
 ('whose', 0.3291302025318146),
 ('marked', 0.32526901364326477)]

In [61]:
data = dataset.loc[:,  

               ["variety","country","province","points","price"]]

data['review']=words
data

,variety,country,province,points,price,review
0,Sparkling Blend,US,California,85,22.0,"[sparkling, wine, texture, mouthfeel, chardonn..."
1,Chardonnay,France,Burgundy,85,15.0,"[fruity, wine, free, oak, aging, revel, attrac..."
2,Cabernet Sauvignon,US,California,90,35.0,"[rich, ripe, complex, cabernet, classic, napa,..."
3,White Blend,Italy,Tuscany,87,18.0,"[cheerful, white, blend, tuscany, colombana, m..."
4,Pinot Noir,New Zealand,Central Otago,87,24.0,"[medium, weight, supple, pinot, noir, bit, oak..."
...,...,...,...,...,...,...
204995,Chardonnay,US,California,91,25.0,"[warmed, croissant, yeastiness, match, sea, sa..."
204996,Syrah,US,Washington,91,48.0,"[ciel, du, cheval, vineyard, fruit, make, majo..."
204997,Syrah,US,Washington,90,27.0,"[chelan, region, washington, newest, ava, alre..."
204998,Chardonnay,US,New York,85,11.0,"[appealing, nose, earthy, spice, start, succes..."


In [62]:
dvariety = data.loc[:,["variety","review"]]
dvariety.shape

(205000, 2)

In [63]:
nvariety = pd.Series(dvariety['variety']).to_numpy()
type(nvariety)

numpy.ndarray

In [64]:
nreviews = pd.Series(dvariety['review']).to_numpy()
type(nreviews)

numpy.ndarray

In [65]:
nvariety= nvariety.astype(str)
nvariety.dtype

dtype('<U35')

In [66]:
train_data= nreviews[:2000]
train_labels= nvariety[:2000]
test_data= nreviews[2000:3000]
test_labels= nvariety[2000:3000]
train_data.shape

(2000,)

Word Mover's Distance:

In [67]:
def dist(rev1, rev2):
  return model.wv.wmdistance(rev1, rev2)

In [68]:
test_review = test_data[1] 
test_label = test_labels[1]

In [69]:
perdicted_label = classify(test_review,6)

In [70]:
print(f"Predicted variety for the first wine is {perdicted_label} and the true label is {test_label}")

Predicted variety for the first wine is Garganega and the true label is Pecorino


In [71]:
test_predicted_n9 = [classify(test_review,9) for test_review in test_data]

In [72]:
n_correct_k9 = np.sum(test_labels == test_predicted_n9, dtype=object)
accuracy_k9 = n_correct_k9/len(test_labels)
print(accuracy_k9)
#Accuracy of the Word Mover's Distance classifier, k=9, in predicting the grape variety (Word2Vec)

0.343


Päris asjani on tükk maad veel. Mu arust võiks jõuda selleni, et saaks veinisoovitusi nii karakterit kirjeldavate märksõnade kui veininime järgi (ehk mis veinid oleksid konkreetsele veinile kõige sarnasemad). Word2veci saab ära kasutada selles mõttes, et märksõna järgi otsides ei pea istuma me kinni ühe kindla sõna küljes (nt kui otsime kuiva veini, siis me ei saa vastu ainult neid, mille kirjelduses on konkreetselt sõna "kuiv"), vaid saame võtta arvesse ka selle sünonüüme/semantiliselt lähedasi sõnu. Siinse katsetuse põhjal saame ehk algatada arutelu?